# fib Model Code 2010: creep and shrinkage

***

<center><i>Petr Havlásek (c) 2023-24, petr.havlasek@cvut.cz</i></center>

***

Implementation of expressions from fib bulletin 65: Model Code 2010, Final draft, Volume 1, 2012.
Relaxation function computed using equations (5.24)-(5.25) presented in Z.P. Bažant and M. Jirásek: Creep and Hygrothermal Effects in Concrete Structures, 2018


<div class="alert alert-block alert-danger">

## Range of applicability

- unreinforced concrete moist cured at normal temperature up to 14 days<br>

- age at loading $tt \geq$ 1 day<br>

- structural concrete with 20 MPa $\leq$ f_cm $\leq$ 130 MPa<br>

- stress level up to 0.4 $f_{cm}(tt)$ for linear creep<br>

- stress level 0.4-0.6 $f_{cm}(tt)$ for nonlinear creep<br>

- mean relative humidity 40-100%<br>

- mean temperature 5-30 C<br>
</div>



## Imports

In [1]:
online = False

if (online):
    import micropip
    await micropip.install('ipywidgets')
    
debug = False

import matplotlib
import matplotlib.pyplot as plt
from IPython.display import display

import math
import numpy as np

import ipywidgets as widgets
from ipywidgets import HBox, VBox


## Definition of Global Variables and Ranges

In [2]:
# drawing - global variables
log_scale = True
draw_toi = False
select_plot = "characteristic functions" # for comparison

# time values and ranges
t_div = 100 # default number of time points

# e.g. shortest loading duration
t_aux_min = 1.e-2 

# concrete age at loading (default value)
tt = 14
tt_min = 1
tt_max = 100
tt_step = 1

# onset of drying (concrete age, default value)
t0 = 7
t0_min = 0
t0_max = 14
t0_step = 1

# plot ranges
t_plot_min = t_aux_min 
t_plot_max = 1.e4

# default time of interest = concrete age
# toi = concrete age
toi = tt + 7.
toi_min = tt
toi_max = 1.e4
toi_step = 7.

# time ranges used in the deformation plot
# here time = concrete age
t_def_min = 1.
t_def_max = t_plot_max

# magnitude of loading stress [MPa]
stress = 10
stress_min = 0
stress_max = 100
stress_step = 5

# mean compressive strength on cylinders at the age of 28 days [MPa]
fcm_28 = 50
fcm_28_min = 20
fcm_28_max = 130
fcm_28_step = 5

# relative humidity of the environment [%] (range of validity 20-100%)
RH = 50
RH_min = 40
RH_max = 100
RH_step = 5

# notional size of the member [mm] (h = 2*cross section / perim. of drying surface)
h = 100
h_min = 50
h_max = 500
h_step = 50


# default value of parameters related to concrete composition
# STIFFNESS-SPECIFIC PARAMETERS
# Table 5.1-6 from MC2010, default value for quartzite/dense limestone
aggregate_type = "quartzite" # default choice
alpha_E = 1.0

# CREEP-SPECIFIC PARAMETERS
cement_type = "CN" # default choice

# value for concrete with fcm below 60 MPa and CN cement
s = 0.25

alpha = 0. #(default values for cement type CN)

# SHRINKAGE-SPECIFIC PARAMETERS, default values for cement type CN
alpha_as = 700.
alpha_ds1 = 4.
alpha_ds2 = 0.012


# mean compressive strength to assess creep linearity
def compute_compressive_strength(tt):

    beta_cc = math.exp( s * (1. - math.sqrt(28./tt) ) ) # 5.1-51
    fcm_t = beta_cc * fcm_28   
    return fcm_t

fcm_t = compute_compressive_strength(tt)  


## Modulus of Elasticity

> Described in Section 5.1.7.2<br>

> $E_{cm}$ is defined as a tangent modulus of elasticity at the origin of the stress-strain diagram which is approximately equal to the slope of the unloading branch at 40% of the compressive strength<br>

> The value of Young's modulus at the age at loading is used to determine the instantaneous deformation while its value at 28 days, $E_{cm}(28)$ or `E_cm_28`, is used to compute the delayed deformation by means of the creep coefficient.


In [3]:
def compute_modulus_of_elasticity(t):

    E_c0 = 21.5e3
    
    E_cm_28 = E_c0 * alpha_E * ( fcm_28/10. )**(1./3.)  # 5.1-21
    
    beta_cc = math.exp( s * (1. - math.sqrt(28./t) ) ) # 5.1-51
    
    beta_E = math.sqrt(beta_cc) # 5.1-57
    
    E_cm_t = beta_E * E_cm_28 # 5.1-56

    return E_cm_t
    

## Creep Coefficient

In [4]:
def compute_creep_coeff(t,tt):
    
    phi_bc = compute_creep_coeff_basic(t, tt)
    phi_dc = compute_creep_coeff_drying(t, tt)
    
    return phi_bc + phi_dc

In [5]:
def compute_creep_coeff_basic(t, tt):

    beta_bc_fcm = 1.8 * fcm_28**(-0.7) # 5.1-65

    # adjusted age reflecting the cement type but (here) without temperature effect
    tt_adj = compute_adjusted_age(tt)

    if (t > tt):
        beta_bc_t_tt = math.log( (30./tt_adj + 0.035)**2. * (t-tt) + 1.) # 5.1-66
    else:
        beta_bc_t_tt = 0.
    
    phi_bc = beta_bc_fcm * beta_bc_t_tt # 5.1-64

    
    if (stress/fcm_t > 0.4): 
        phi_corr = compute_nonlinear_creep_correction()
        phi_bc *= phi_corr
    
    return phi_bc
    

In [6]:
def compute_creep_coeff_drying(t, tt):

    # adjusted age reflecting the cement type but (here) without temperature effect
    tt_adj = compute_adjusted_age(tt)
    #-------------------
    beta_dc_fcm = 412. * fcm_28**(-1.4) # 5.1-68
    #-------------------
    beta_RH = (1. - RH/100.) * (0.1 * h/100.)**(-1./3.) # 5.1-69
    #-------------------
    beta_dc_tt = 1. / (0.1 + tt_adj**0.2) # 5.1-70
    #-------------------
    gamma_tt = 1. / ( 2.3 + ( 3.5 / math.sqrt(tt_adj) ) ) # 5.1-71b

    alpha_fcm = math.sqrt(35./fcm_28) # 5.1-71d
    
    beta_h = min(1.5 * h + 250.*alpha_fcm, 1500 * alpha_fcm) # 5.1-71c
    
    beta_dc_t_tt = ( (t-tt) / ( beta_h + (t-tt) ) )**gamma_tt # 5.1-71a
    #-------------------                 
    phi_dc = beta_dc_fcm * beta_RH * beta_dc_tt * beta_dc_t_tt # 5.1-67

    if (stress/fcm_t > 0.4): 
        phi_corr = compute_nonlinear_creep_correction()
        phi_dc *= phi_corr
                    
    return phi_dc


In [7]:
def compute_adjusted_age(tt):
    # without temperature effect
    tt_adj = max( tt * ( 9./ ( 2. + tt**1.2 ) + 1. )**alpha, 0.5) # 5.1-73

    return tt_adj
    

Creep enhancement at high stress levels between 40% and 60% $f_{cm}(t)$

In [8]:
def compute_nonlinear_creep_correction():

    # section 5.1-74
    if (stress/fcm_t < 0.4):
        correction = 1.
    else:
        correction = math.exp(1.5 * ( stress/fcm_t  - 0.4) )

    return correction
    

## Compliance and Relaxation Functions

In [9]:
def compute_total_compliance_function(t, tt):

    if (t < tt):
        J = 0.
        
    elif ( t == tt):
        E_cm_t = compute_modulus_of_elasticity(tt)
        J = 1./E_cm_t
        
    else:
        E_cm_t = compute_modulus_of_elasticity(tt)
        E_cm_28 = compute_modulus_of_elasticity(28.)
        
        # 5.1-63
        # sum of basic and drying creep coefficient
        phi = compute_creep_coeff(t,tt)
        
        # 5.1.61 
        J = 1./E_cm_t + phi/E_cm_28
    
    return J

def compute_basic_compliance_function(t, tt):

    if (t < tt):
        J = 0.
        
    elif ( t == tt):
        E_cm_t = compute_modulus_of_elasticity(tt)
        J = 1./E_cm_t
        
    else:
        E_cm_t = compute_modulus_of_elasticity(tt)
        E_cm_28 = compute_modulus_of_elasticity(28.)
        
        # 5.1-63
        phi = compute_creep_coeff_basic(t, tt) 
        
        # 5.1.61 
        J = 1./E_cm_t + phi/E_cm_28
    
    return J


In [10]:
def compute_total_relaxation_function(t, tt):

    if (t < tt):
        R = 0.

    elif (t == tt):
        R = compute_modulus_of_elasticity(tt)

    else:
        R_tot = np.zeros(t_div)
        
        # create array with loading durations [0., 0., logscale with t_div-2 entries]
        if (math.log10(t-tt)) > round(math.log10(t_aux_min))-1:
            times = np.logspace( round(math.log10(t_aux_min))-1, ( math.log10(t-tt) ), num = t_div-2 )
        else: # very short loading durations
            times = np.logspace( ( math.log10(t-tt) )-2, ( math.log10(t-tt) ), num = t_div-2 )
        times = np.insert( times, 0, [0., 0.])
        
        times += tt # duration of loading changed to age

        # zero loading duration, times[1] = times[0]
        R_tot[1] = 1. / compute_total_compliance_function(times[1], times[0]) 
        # first non-zero loading duration
        R_tot[2] = 1. / compute_total_compliance_function(times[2], times[0])
   
        for k in range(2,t_div-1):
            
            sum = 0.
            for i in range(0,k-1):
                J_kplus1_i = compute_total_compliance_function( times[k+1], (times[i+1]+times[i])/2. ) 
                J_k_i = compute_total_compliance_function( times[k], (times[i+1]+times[i])/2. ) 
                delta_J_k_i = J_kplus1_i - J_k_i
                sum += delta_J_k_i * ( R_tot[i+1] - R_tot[i] )
    
            J_kplus1_k = compute_total_compliance_function( times[k+1], (times[k+1]+times[k])/2. )
            R_tot[k+1] = R_tot[k] - sum / J_kplus1_k
        
        # return the last value from the array which corresponds to the desired time "t"
        R = R_tot[-1]
    
    return R

## Mean Total Shrinkage

5.1.9.4.4: characteristic values corresponding to 5 and 95 percentile are 0.42 eps_cs and 1.58 eps_cs, respectively

In [11]:
def compute_mean_total_shrinkage(t):

    eps_cas = compute_mean_auto_shrinkage(t)
    eps_cds = compute_mean_drying_shrinkage(t)
    eps_cs = eps_cas + eps_cds # 5.1-75

    return eps_cs
    

### Mean Autogenous Shrinkage

In [12]:
def compute_mean_auto_shrinkage(t):

    if (t <= 0.):
        eps_cas = 0.
    else:
        # 5.1-78, notional autogenous shrinkage coefficient
        eps_cas_fcm = -alpha_as * ( fcm_28 / ( 60. + fcm_28 ) )**2.5 * 1.e-6
        # 5.1-79, development of autogenous shrinkage
        beta_as_t = 1. - math.exp( -0.2 * math.sqrt(t) )
        
        eps_cas = eps_cas_fcm * beta_as_t # 5.1-76

    return eps_cas

### Mean Drying Shrinkage

In [13]:
def compute_mean_drying_shrinkage(t):

    t_dry = t-t0
    
    if (t_dry <= 0.):
        eps_cds = 0.
    
    else:       
        # 5.1-80, notional drying shrinkage coefficient
        eps_cds_fcm = (220. + 110 * alpha_ds1) * math.exp(-alpha_ds2 * fcm_28) * 1.e-6
    
    
        # 5.1-83
        beta_s1 = min( (35./fcm_28)**0.1, 1.0 )
        
        # 5.1-81 effect of relative humidity on drying shrinkage
        if (RH < 99. * beta_s1): 
            beta_RH = -1.55 * (1. - (RH/100.)**3 )
        else: 
            beta_RH =  0.25
    
        # 5.1-82, development of drying shrinkage
        beta_DS_t_dry = math.sqrt( t_dry / (0.035 * h**2  + t_dry) )
    
        eps_cds = eps_cds_fcm * beta_RH * beta_DS_t_dry # 5.1-77

    return eps_cds

## Plotting

### Plotting init and updating

In [14]:
output = widgets.Output()

# compliance and relaxation functions
fig, ax = plt.subplots(1, 2, figsize=(10,5))
plt.rcParams.update({'font.size': 14})
plt.close(fig)

# deformation 
fig2, ax2 = plt.subplots(1, 1, figsize=(10,5))
plt.rcParams.update({'font.size': 14})
plt.close(fig2)

# deformation - confidence intervals
fig3, ax3 = plt.subplots(1, 2, figsize=(10,5))
plt.rcParams.update({'font.size': 14})
plt.close(fig3)


def update_output():
    with output:

        output.clear_output(wait = True)

        # Results with compliance funtion evaluated at TOI
        J_toi = compute_total_compliance_function(toi, tt)
        J_toi_label.value = f"   J = {(J_toi):.4e} 1/MPa"

        # Relaxation function evaluated at TOI
        R_toi = compute_total_relaxation_function(toi, tt)
        R_toi_label.value = f"   R = {(R_toi):.4e} MPa"


        if (select_plot=="characteristic functions"):
            plot_MC2010_compliance_function()
            plot_MC2010_relaxation_function()
            display(fig)
        
        elif (select_plot=="strain decomposition"):
            plot_total_deformation()
            display(fig2)

        else: # confidence intervals
            plot_total_deformation_confidence()
            display(fig3)
        
   


### Plot of a compliance function 

In [15]:
def plot_MC2010_compliance_function():       

    # times here is the duration of loading
    if (log_scale):
        times = np.logspace( round(math.log10(t_plot_min)), round(math.log10(t_plot_max)), num = t_div )
    else:
        times = np.linspace(0., t_plot_max, num = t_div )

    ax[0].clear()
    
    if (stress/fcm_t > 0.6):
        ax[0].text(0.5, 0.5, 'unsupported stress level!', bbox=dict(facecolor='red', alpha=0.5), fontsize=20, horizontalalignment='center', verticalalignment='center', transform=ax[0].transAxes)
        return
    elif (stress/fcm_t > 0.4):
        ax[0].text(0.5, 0.5, 'nonlinear creep!', bbox=dict(facecolor='red', alpha=0.5), fontsize=20, horizontalalignment='center', verticalalignment='center', transform=ax[0].transAxes)
    else:
        pass
       
        
    J_ini = np.zeros(t_div)
    J_bas = np.zeros(t_div)
    J_tot = np.zeros(t_div)

    E_cm_t = compute_modulus_of_elasticity(tt)
        
    for i in range(t_div):
        # here t is shifted by the onset of loading
        t = times[i] + tt
        J_ini[i] =  1./E_cm_t 
        J_bas[i] =  compute_basic_compliance_function(t, tt) 
        #J_bas[i] =  1./E_cm_t + compute_creep_coeff_basic(t, tt)/E_cm_28 
        J_tot[i] =  compute_total_compliance_function(t, tt) 
                
    ax[0].plot(times, J_ini * 1.e6, lw=1., color="cyan", linestyle='dashed', label="initial")
    ax[0].plot(times, J_bas * 1.e6, lw=2., color="blue", label=r"$J_b$")
    ax[0].plot(times, J_tot * 1.e6, lw=2., color="black", label=r"$J_{tot}$")

    # plot vertical line corresponding to time of interest
    if ( draw_toi ):
        if (toi - tt) > 0.:
            ax[0].axvline(x = toi - tt, lw=2., color="green", linestyle = "--")
    
    if (log_scale):
        ax[0].set_xscale('log')
        ax[0].set_xlim([t_plot_min, t_plot_max])
    else:
        ax[0].set_xlim([0., t_plot_max])
 
    # keep top range unbounded
    ax[0].set_ylim(bottom = 0)
    
    ax[0].legend()
    ax[0].set_xlabel('Duration of loading, $t-t\'$ [day]', fontsize=14)
    ax[0].set_ylabel('Compliance, $J$ [$10^{-6}$/MPa]', fontsize=14)

    ax[0].grid(True)



### Plot of a relaxation function 

In [16]:
def plot_MC2010_relaxation_function():        
    # =======================
    # RELAXATION FUNCTION

    # create array with loading durations [0., 0., logscale with t_div-2 entries]
    times = np.logspace( round(math.log10(t_aux_min))-1, round(math.log10(t_plot_max)), num = t_div-2 )
    times = np.insert( times, 0, [0., 0.])

    times += tt # duration of loading changed to age
    
    ax[1].clear()


    if (stress/fcm_t > 0.6):
        ax[1].text(0.5, 0.5, 'unsupported stress level!', bbox=dict(facecolor='red', alpha=0.5), fontsize=20, horizontalalignment='center', verticalalignment='center', transform=ax[1].transAxes)
        return
    elif (stress/fcm_t > 0.4):
        ax[1].text(0.5, 0.5, 'nonlinear creep!', bbox=dict(facecolor='red', alpha=0.5), fontsize=20, horizontalalignment='center', verticalalignment='center', transform=ax[1].transAxes)
    else:
        pass

    R_ini = np.zeros(t_div)
    R_bas = np.zeros(t_div)
    R_tot = np.zeros(t_div)


    R_ini += compute_modulus_of_elasticity(tt)

    #--------------------------------------------------

    # zero loading duration, times[1] = times[0]
    R_bas[1] = 1. / compute_basic_compliance_function(times[1], times[0]) 
    # first non-zero loading duration
    R_bas[2] = 1. / compute_basic_compliance_function(times[2], times[0])

    for k in range(2,t_div-1):
        
        sum = 0.
        for i in range(0,k-1):
            J_kplus1_i = compute_basic_compliance_function( times[k+1], (times[i+1]+times[i])/2. ) 
            J_k_i = compute_basic_compliance_function( times[k], (times[i+1]+times[i])/2. ) 
            delta_J_k_i = J_kplus1_i - J_k_i
            sum += delta_J_k_i * ( R_bas[i+1] - R_bas[i] )

        J_kplus1_k = compute_basic_compliance_function( times[k+1], (times[k+1]+times[k])/2. )
        R_bas[k+1] = R_bas[k] - sum / J_kplus1_k

    #--------------------------------------------------

    # zero loading duration, times[1] = times[0]
    R_tot[1] = 1. / compute_total_compliance_function(times[1], times[0]) 
    # first non-zero loading duration
    R_tot[2] = 1. / compute_total_compliance_function(times[2], times[0])
        
    for k in range(2,t_div-1):
        
        sum = 0.
        for i in range(0,k-1):
            J_kplus1_i = compute_total_compliance_function( times[k+1], (times[i+1]+times[i])/2. ) 
            J_k_i = compute_total_compliance_function( times[k], (times[i+1]+times[i])/2. ) 
            delta_J_k_i = J_kplus1_i - J_k_i
            sum += delta_J_k_i * ( R_tot[i+1] - R_tot[i] )

        J_kplus1_k = compute_total_compliance_function( times[k+1], (times[k+1]+times[k])/2. )
        R_tot[k+1] = R_tot[k] - sum / J_kplus1_k
   
    #--------------------------------------------------
            
    # plot vertical line corresponding to time of interest
    if ( draw_toi ):
        if (toi - tt) > 0.:
            ax[1].axvline(x = toi - tt, lw=2., color="green", linestyle = "--")
    
    if (log_scale):

        ax[1].plot(times[2:]-tt, R_ini[2:] * 1.e-3, lw=1., color="cyan", linestyle='dashed', label="initial")
        ax[1].plot(times[2:]-tt, R_bas[2:] * 1.e-3, lw=2., color="blue", label=r"$R_b$")
        ax[1].plot(times[2:]-tt, R_tot[2:] * 1.e-3, lw=2., color="black", label=r"$R_{tot}$")
        
        ax[1].set_xscale('log')
        ax[1].set_xlim([t_plot_min, t_plot_max])
    
    else: # linscale

        ax[1].plot(times[1:]-tt, R_ini[1:] * 1.e-3, lw=1., color="cyan", linestyle='dashed', label="initial")
        ax[1].plot(times[1:]-tt, R_bas[1:] * 1.e-3, lw=2., color="blue", label=r"$R_b$")
        ax[1].plot(times[1:]-tt, R_tot[1:] * 1.e-3, lw=2., color="black", label=r"$R_{tot}$")
    
        ax[1].set_xlim([0., t_plot_max])

    # keep top range unbounded
    ax[1].set_ylim(bottom = 0)
    
    ax[1].legend()
    ax[1].set_xlabel('Duration of loading, $t-t\'$ [day]', fontsize=14)
    ax[1].set_ylabel('Relaxation, $R$ [GPa]', fontsize=14)

    ax[1].grid(True)


### Plot of total deformation with strain decomposition

In [17]:

def plot_total_deformation():       

    # times here are the actual concrete age
    if (log_scale):
        times = np.logspace( round(math.log10(t_def_min)), round(math.log10(t_def_max)), num = t_div )
    else:
        times = np.linspace(t_def_min, t_plot_max, num = t_div )

    times = np.append(times, [tt-t_aux_min, tt, tt+t_aux_min])
    times = np.append(times, [t0+t_aux_min, t0, t0+t_aux_min])
    # eliminates redundancy and sorts
    times = np.unique(times)
    
    # local variable redifinition only
    len_times = len(times)

    ax2.clear()


    if (stress/fcm_t > 0.6):
        ax2.text(0.5, 0.5, 'unsupported stress level!', bbox=dict(facecolor='red', alpha=0.5), fontsize=20, horizontalalignment='center', verticalalignment='center', transform=ax2.transAxes)
        return
    elif (stress/fcm_t > 0.4):
        ax2.text(0.5, 0.5, 'nonlinear creep!', bbox=dict(facecolor='red', alpha=0.5), fontsize=20, horizontalalignment='center', verticalalignment='center', transform=ax2.transAxes)
    else:
        pass
        
    creep_bas = np.zeros(len_times)
    creep_tot = np.zeros(len_times)

    shr_auto = np.zeros(len_times)
    shr_tot = np.zeros(len_times)

    def_tot = np.zeros(len_times)

    E_cm_t = compute_modulus_of_elasticity(tt)
        
    for i in range(len_times):
        # here t is shifted by the onset of loading
        t = times[i]

        shr_auto[i] = compute_mean_auto_shrinkage(t) * 1.e6
        shr_tot[i] = compute_mean_total_shrinkage(t) * 1.e6
        
        if (t >= tt):
            creep_bas[i] =  stress * compute_basic_compliance_function(t, tt) * -1.e6 
            creep_tot[i] =  stress * compute_total_compliance_function(t, tt) * -1.e6 

        def_tot[i] = creep_tot[i] + shr_tot[i]
                
    
    ax2.plot(times, creep_bas, lw=2., color="blue", label=r"Basic creep")
    ax2.plot(times, creep_tot, lw=2., color="black", label=r"Total creep")

    ax2.plot(times, shr_auto, lw=2., color="blue", label=r"Autogenous shrinkage", linestyle='dashed')
    ax2.plot(times, shr_tot, lw=2., color="black", label=r"Total shrinkage", linestyle='dashed')

    ax2.plot(times, def_tot, lw=2., color="red", label=r"Total deformation")

    # plot vertical line corresponding to time of interest
    if ( draw_toi ):
        ax2.axvline(x = toi, lw=2., color="green", linestyle = "--")
    
    if (log_scale):
        ax2.set_xscale('log')
        ax2.set_xlim([t_def_min, t_def_max])
    else:
        ax2.set_xlim([0., t_def_max])
    
    ax2.legend()
    ax2.set_xlabel('Concrete age, $t$ [day]', fontsize=14)
    ax2.set_ylabel('Strain, $\epsilon$ [$10^{-6}$]', fontsize=14)

    ax2.grid(True)


### Plot creep and shrinkage deformation with confidence intervals

In [18]:

def plot_total_deformation_confidence():       

    # times here are the actual concrete age
    if (log_scale):
        times = np.logspace( round(math.log10(t_def_min)), round(math.log10(t_def_max)), num = t_div )
    else:
        times = np.linspace(t_def_min, t_plot_max, num = t_div )

    times = np.append(times, [tt-t_aux_min, tt, tt+t_aux_min])
    times = np.append(times, [t0+t_aux_min, t0, t0+t_aux_min])
    # eliminates redundancy and sorts
    times = np.unique(times)
    
    # local variable redifinition only
    len_times = len(times)

    ax3[0].clear()
    ax3[1].clear()

    if (stress/fcm_t > 0.6):
        ax3[0].text(0.5, 0.5, 'unsupported stress level!', bbox=dict(facecolor='red', alpha=0.5), fontsize=20, horizontalalignment='center', verticalalignment='center', transform=ax3.transAxes)
        return
    elif (stress/fcm_t > 0.4):
        ax3[0].text(0.5, 0.5, 'nonlinear creep!', bbox=dict(facecolor='red', alpha=0.5), fontsize=20, horizontalalignment='center', verticalalignment='center', transform=ax3.transAxes)
    else:
        pass

    creep_tot = np.zeros(len_times)
    creep_tot_05 = np.zeros(len_times)
    creep_tot_10 = np.zeros(len_times)
    creep_tot_90 = np.zeros(len_times)
    creep_tot_95 = np.zeros(len_times)

    shr_tot = np.zeros(len_times)
    shr_tot_05 = np.zeros(len_times)
    shr_tot_10 = np.zeros(len_times)
    shr_tot_90 = np.zeros(len_times)
    shr_tot_95 = np.zeros(len_times)

    E_cm_t = compute_modulus_of_elasticity(tt)
    E_cm_28 = compute_modulus_of_elasticity(28.)
        
    for i in range(len_times):
        # here t is shifted by the onset of loading
        t = times[i]

        shr_tot[i] = compute_mean_total_shrinkage(t) * 1.e6

        shr_tot_05[i] = 0.42 * compute_mean_total_shrinkage(t) * 1.e6
        shr_tot_10[i] = 0.55 * compute_mean_total_shrinkage(t) * 1.e6
        shr_tot_90[i] = 1.45 * compute_mean_total_shrinkage(t) * 1.e6
        shr_tot_95[i] = 1.58 * compute_mean_total_shrinkage(t) * 1.e6
        
        if (t >= tt):
            phi = compute_creep_coeff(t,tt)
            creep_tot[i] =  stress * ( 1./E_cm_t + phi/E_cm_28 ) * -1.e6 

            creep_tot_05[i] = stress * ( 1./E_cm_t + 0.59 * phi / E_cm_28 ) * -1.e6 
            creep_tot_10[i] = stress * ( 1./E_cm_t + 0.68 * phi / E_cm_28 ) * -1.e6 
            creep_tot_90[i] = stress * ( 1./E_cm_t + 1.32 * phi / E_cm_28 ) * -1.e6 
            creep_tot_95[i] = stress * ( 1./E_cm_t + 1.41 * phi / E_cm_28 ) * -1.e6 
                
    
    ax3[0].plot(times, creep_tot, lw=2., color="black", label=r"Total creep")
    ax3[0].plot(times, creep_tot_05, lw=1., color="blue", label =r"05/95", linestyle='dashed')
    ax3[0].plot(times, creep_tot_95, lw=1., color="blue", linestyle='dashed')
    ax3[0].plot(times, creep_tot_10, lw=1.5, color="grey", label =r"10/90", linestyle='dashed')
    ax3[0].plot(times, creep_tot_90, lw=1.5, color="grey", linestyle='dashed')

    ax3[1].plot(times, shr_tot, lw=2., color="black", label=r"Total shrinkage", linestyle='dashed')
    ax3[1].plot(times, shr_tot_05, lw=1., color="blue", label =r"05/95", linestyle='dashed')
    ax3[1].plot(times, shr_tot_95, lw=1., color="blue", linestyle='dashed')
    ax3[1].plot(times, shr_tot_10, lw=1.5, color="grey", label =r"10/90", linestyle='dashed')
    ax3[1].plot(times, shr_tot_90, lw=1.5, color="grey", linestyle='dashed')

   
    if (log_scale):
        ax3[0].set_xscale('log')
        ax3[0].set_xlim([t_def_min, t_def_max])

        ax3[1].set_xscale('log')
        ax3[1].set_xlim([t_def_min, t_def_max])
    else:
        ax3[0].set_xlim([0., t_def_max])

        ax3[1].set_xlim([0., t_def_max])
    
    ax3[0].legend()
    ax3[0].set_xlabel('Concrete age, $t$ [day]', fontsize=14)
    ax3[0].set_ylabel('Strain, $\epsilon$ [$10^{-6}$]', fontsize=14)

    ax3[0].grid(True)

    ax3[1].legend()
    ax3[1].set_xlabel('Concrete age, $t$ [day]', fontsize=14)
    
    ax3[1].grid(True)

## GUI
### Definition of widgets

In [19]:

# this style enables longer description
style = {'description_width': 'initial'}

## LOADING & CURING & CONDITIONS
# age at loading
tt_box = widgets.BoundedFloatText( value=tt, min=tt_min, max = tt_max, step = tt_step, description='age at loading, tt [day]', disabled=False, style=style)

# onset of drying
t0_box = widgets.BoundedFloatText( value=t0, min=t0_min, max = t0_max, step = t0_step, description='duration of curing, t0 [day]', disabled=False, style=style)

# stress
stress_box = widgets.BoundedFloatText( value=stress, min=stress_min, max=stress_max, step=stress_step, description='stress (compression) [MPa]', disabled=False, style=style)

# ambient relative humidity
RH_box = widgets.BoundedIntText( value=RH, min=RH_min, max = RH_max, step = RH_step, description='ambient relative humidity [%]', disabled=False, style=style)

## MATERIAL PROPERTIES & GEOMETRY
# mean compressive strength on cylinders at the age of 28 days [MPa]
fcm_box = widgets.BoundedIntText( value=fcm_28, min=fcm_28_min, max = fcm_28_max, step = fcm_28_step, description='strength, fcm28 [MPa]', disabled=False, style=style)

# notional specimen size [mm]
h_box = widgets.BoundedIntText( value=h, min=h_min, max = h_max, step = h_step, description='notional size, h [mm]', disabled=False, style=style)

## selection - aggregate & cement
aggregate_type_select = widgets.Dropdown( options=['basalt', 'quartzite', 'dense limestone', 'limestone', 'sandstone'],
    value=aggregate_type, description='aggregate type:', style=style)

cement_type_select = widgets.Dropdown( options=['CS', 'CN', 'CR'], value=cement_type, description='cement type:', style=style)

# checkboxes
log_scale_checkbox = widgets.Checkbox(value=log_scale, description='log-scale', style=style)

draw_toi_checkbox = widgets.Checkbox(value=draw_toi, description='draw time of interest', style=style)

select_plot_select = widgets.Dropdown(value=select_plot, options=['characteristic functions', 'strain decomposition', 'confidence intervals'], description='plot selection', style=style)

toi_box = widgets.BoundedFloatText( value=toi, min=toi_min, max=toi_max, step=toi_step, description='time of interest [day]', disabled=False, style=style)


J_toi = 0.
J_toi_label = widgets.Label(value=str(J_toi))

R_toi = 0.
R_toi_label = widgets.Label(value=str(R_toi))


### Widget events

In [20]:
def on_tt_change(change):
    global tt
    tt = change['new']

    '''
    if (tt > toi):
        toi_box.value = tt + t_aux_min
    '''

    global fcm_t
    fcm_t = compute_compressive_strength(tt)  
    
    update_output()   

def on_t0_change(change):
    global t0
    t0 = change['new']
    
    update_output()   

def on_stress_change(change):
    global stress
    stress = change['new']
    global fcm_t
    fcm_t = compute_compressive_strength(tt)  
   
    update_output()

def on_RH_change(change):
    global RH
    RH = change['new']
    update_output()       


def on_fcm_change(change):
    global fcm_28
    fcm_28 = change['new']

    global fcm_t
    fcm_t = compute_compressive_strength(tt)  
    
    update_output()    

def on_h_change(change):
    global h
    h = change['new']
    update_output()        

def on_cement_type_change(change):

    cement_type = change['new']

    # STIFFNESS and strength development
    # table 5.1-9 (page 139)
    global s

    if (fcm_28 <= 60): # MPa
    
        if (cement_type == "CS"):
            s = 0.38

        elif (cement_type == "CN"):
            s = 0.25

        else: # R
            s = 0.20

    else:
        s = 0.20

    # CREEP
    # effect of cement type on adjusted time, page 145
    global alpha

    if (cement_type == "CS"):
        alpha = -1.

    elif (cement_type == "CN"):
        alpha = 0.

    else: # CR
        alpha = 1.
        
    # SHRINKAGE 
    global alpha_as, alpha_ds1, alpha_ds2

    # values from Table 5.1-12
    if (cement_type == "CS"):
        alpha_as = 800.
        alpha_ds1 = 3.
        alpha_ds2 = 0.013

    elif (cement_type == "CN"):
        alpha_as = 700.
        alpha_ds1 = 4.
        alpha_ds2 = 0.012

    else: # CR
        alpha_as = 600.
        alpha_ds1 = 6.
        alpha_ds2 = 0.012

    update_output()   

def on_aggregate_type_change(change):

    aggregate_type = change['new']
    
    global alpha_E  
    
    # Table 5.1-6 from MC2010   
    if (aggregate_type == "basalt"):
        alpha_E = 1.2
        
    elif (aggregate_type == "quartzite"): 
        alpha_E = 1.0

    elif (aggregate_type == "dense limestone"): 
        alpha_E = 1.0

    elif (aggregate_type == "limestone"):
        alpha_E = 0.9

    else: #(aggregate_type == "sandstone")
        alpha_E = 0.7
    
    update_output()   

def on_logscale_change(change):

    global log_scale
    log_scale = change['new']
    update_output()   

def on_draw_toi_change(change):

    global draw_toi
    draw_toi = change['new']
    update_output()   

def on_select_plot_change(change):

    global select_plot
    select_plot = change['new']
    update_output()   

def on_toi_change(change):

    global toi
    toi = change['new']

    '''
    if (toi < tt):
        toi = tt + t_aux_min
        toi_box.value = toi
    '''
   
    update_output()   
  

tt_box.observe(on_tt_change, names = 'value')
t0_box.observe(on_t0_change, names = 'value')
stress_box.observe(on_stress_change, names = 'value')
RH_box.observe(on_RH_change, names = 'value')
fcm_box.observe(on_fcm_change, names = 'value')
h_box.observe(on_h_change, names = 'value')
aggregate_type_select.observe(on_aggregate_type_change, names = 'value')
cement_type_select.observe(on_cement_type_change, names = 'value')
log_scale_checkbox.observe(on_logscale_change, names = 'value')
draw_toi_checkbox.observe(on_draw_toi_change, names = 'value')
select_plot_select.observe(on_select_plot_change, names = 'value')
toi_box.observe(on_toi_change, names = 'value')


### LAYOUT

In [21]:

opt_mat = widgets.HBox([ widgets.VBox([fcm_box, aggregate_type_select, cement_type_select]),  widgets.VBox([h_box, t0_box])])
opt_load = widgets.VBox([ RH_box, widgets.HBox([tt_box, stress_box]) ])
opt_res = widgets.HBox([ widgets.VBox([tt_box, toi_box]), widgets.VBox([J_toi_label, R_toi_label])])
opt_plot = widgets.VBox([  widgets.HBox([log_scale_checkbox, draw_toi_checkbox]), select_plot_select])

accordion = widgets.Accordion(children=[opt_mat, opt_load, opt_res, opt_plot], titles=('Material & Geometry', 'Loading', 'Results', 'Plot Setup'))


## Results

In [22]:
update_output()

display(output, accordion)

Output()

Accordion(children=(HBox(children=(VBox(children=(BoundedIntText(value=50, description='strength, fcm28 [MPa]'…

<div class="alert alert-block alert-info">

## Input parameters

### Material & Geometry
`fcm28` ... mean compressive strength of concrete measured on cylinders at the age of 28 days [MPa] <br>

`h` ... notional size [mm] reflecting cross section geometry and surface exposed to drying<br>
$h = 2 \times A_{cs} / L_{dry}$<br>
where<br>
$A_{cs} \dots$ cross-section area<br>
$L_{dry} \dots$ cross-section perimeter exposed to drying<br>

`aggregate type` ... basalt/quartzite/dense limestone/sandstone <br>

'cement type' ... (decision tree = first applicable criterion according to the composition)

* CS = CEM III, CEM II/B, strength class 32.5 N, 42.5 N, more than 65% of GGBS or more than 35% FA<br>

* CN = CEM II, CEM I, strength class 42.5 N, 32.5 R, more than 35% GGBS or more than 20 % FA<br>

* CR = CEM I, strength class 42.5 R, 52.5 N, 52.5 R

`t0` ... duration of curing = concrete age at the onset of drying<br>

### Loading

`RH` ... mean relative humidity of the environment [%]<br>

`tt` ... age at loading [day]<br>

`stress` ... magnitude of compression [MPa]<br>


### Results

`time of interest`, `toi` ... user-defined concrete age when the compliance and relaxation functions are evaluated<br>

`J` and `R` ... values of compliance and relaxation functions evaluated at `toi` for the user-specified material definition and loading conditions


### Plotting

`log-scale` ... scale of the time-axis<br>

`draw time of interest` ... shows user-defined time of interest, `toi`<br>

`plot selection`:

* `characteristic functions`: compliance and relaxation functions plotted against duration of loading, $t-tt$

* `strain decomposition`: evolution of shrinkage and creep strains plotted against concrete age

* `confidence intervals`: 5, 10, 90 and 95% confidence limits for creep and shrinkage 







</div>
